In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib
import os

In [2]:
ROOT='../DATA/image/'
if not os.path.exists(ROOT):
    os.makedirs(ROOT)
else: print('already image')

already image


In [ ]:
transform=transforms.Compose([transforms.Resize((256,256),interpolation=transforms.InterpolationMode.BILINEAR),transforms.ToTensor(),transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
train_dataset=CIFAR10(root=ROOT,transform=transform)
test_dataset=CIFAR10(root=ROOT,train=False,transform=transform)
train_dataloader=DataLoader(train_dataset,batch_size=1000,drop_last=True)
test_dataloader=DataLoader(test_dataset,batch_size=1000,drop_last=True)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.con1=nn.Sequential(nn.Conv2d(3,32,3,1,1),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(2,2))
        self.con2=nn.Sequential(
        nn.Conv2d(32,32,3,1),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(2,2))
        self.fc=nn.Sequential(nn.Linear(32*63*63,256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256,64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Linear(64,10)
        )
    def forward(self,data):
        out=self.con1(data)
        out=self.con2(out)
        out=out.view(out.size(0),-1)
        return self.fc(out)

In [31]:
from torchmetrics.classification import Accuracy
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model=CNN()
model.to(DEVICE)
optimizer=optim.Adam(model.parameters(),lr=0.0001)
LR=ReduceLROnPlateau(optimizer=optimizer,mode='min',patience=10)
accuracy=Accuracy(task='multiclass',num_classes=10)
for epoch in range(100):
    model.train()
    loss_list=[]
    pred_list=[]
    fact_list=[]
    for x,y in train_dataloader:
        x,y=x.to(DEVICE),torch.LongTensor(y).to(DEVICE)
        optimizer.zero_grad()
        output=model(x)
        loss=F.cross_entropy(output,y)
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())       
        pred=torch.argmax(output,dim=1).tolist()
        pred_list.extend(pred)
        fact_list.extend(y.tolist())
    accur=accuracy(torch.LongTensor(pred_list),torch.LongTensor(fact_list))
    print(f'epoch: {epoch}')
    print(f'[train] 정확도: {accur}, loss: {sum(loss_list)/len(loss_list)}')  
    with torch.no_grad():
        model.eval()
        loss_list=[]
        pred_list=[]
        fact_list=[]
        for x,y in test_dataloader:
            x,y=x.to(DEVICE),torch.LongTensor(y).to(DEVICE)
            output=model(x)
            loss=F.cross_entropy(output,y)
            loss_list.append(loss.item())       
            pred=torch.argmax(output,dim=1).tolist()
            pred_list.extend(pred)
            fact_list.extend(y.tolist())
        accur=accuracy(torch.LongTensor(pred_list),torch.LongTensor(fact_list))
        print(f'[test] 정확도: {accur}, loss: {sum(loss_list)/len(loss_list)}') 
        total_loss=(sum(loss_list)/len(loss_list))
        LR.step(total_loss)
        

epoch: 0


TypeError: sum(): argument 'input' (position 1) must be Tensor, not list